Version: 02.14.2023

# Challenge Lab 6.3: Implementing Topic Modeling

In this lab, you will use either Amazon Comprehend or the Amazon SageMaker Neural Topic Model (NTM) to extract topics from the [CMU Movie Summary Corpus](http://www.cs.cmu.edu/~ark/personas/). 

## CMU Movie Summary Corpus

The CMU Movie Summary Corpus is a collection of 42,306 movie plot summaries and metadata at both the movie level (including box office revenue, genre, and date of release) and character level (including gender and estimated age).  This data supports work in the following paper:

David Bamman, Brendan O'Connor, and Noah Smith. "Learning Latent Personas of Film Characters." Presented at the Annual Meeting of the Association for Computational Linguistics (ACL 2013), Sofia, Bulgaria, August 2013. http://www.cs.cmu.edu/~dbamman/pubs/pdf/bamman+oconnor+smith.acl13.pdf.

You will use two datasets in this lab:

**plot_summaries.txt**

This dataset contains plot summaries of 42,306 movies, extracted from the November 2, 2012 dump of English-language Wikipedia. Each line contains the Wikipedia movie ID (which indexes into movie.metadata.tsv) followed by the summary.

**movie.metadata.tsv**

This dataset contains metadata for 81,741 movies, extracted from the November 4, 2012 dump of Freebase. The data is tab-separated and contains the following columns:

1. Wikipedia movie ID
2. Freebase movie ID
3. Movie name
4. Movie release date
5. Movie box office revenue
6. Movie runtime
7. Movie languages (Freebase ID:name tuples)
8. Movie countries (Freebase ID:name tuples)
9. Movie genres (Freebase ID:name tuples)

## Lab steps

To complete this lab, you will follow these steps:

1. [Installing the packages](#1.-Installing-the-packages)
2. [Reviewing the dataset](#2.-Reviewing-the-dataset)
3. [Extracting topics](#3.-Extracting-topics)

## Submitting your work

1. In the lab console, choose **Submit** to record your progress and when prompted, choose **Yes**.

1. If the results don't display after a couple of minutes, return to the top of these instructions and choose **Grades**.

     **Tip**: You can submit your work multiple times. After you change your work, choose **Submit** again. Your last submission is what will be recorded for this lab.

1. To find detailed feedback on your work, choose **Details** followed by **View Submission Report**.

## 1. Installing the packages
([Go to top](#Challenge-Lab-6.3:-Implementing-Topic-Extraction))

First, update and install the packages that you will use in the notebook.

In [1]:
%matplotlib inline

import boto3
import os, io, struct, json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import uuid
from time import sleep
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

Matplotlib is building the font cache; this may take a moment.
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...


In [2]:
bucket = "c100688a2296030l5426874t1w241840010076-labbucket-4068hmqh9l3w"
job_data_access_role = 'arn:aws:iam::241840010076:role/service-role/c100688a2296030l5426874t1w-ComprehendDataAccessRole-Am4ndZ38wCo2'
prefix='lab63'

## 2. Reviewing the dataset
([Go to top](#Challenge-Lab-6.3:-Implementing-Topic-Extraction))

First, load the plot_summaries.tsv data into a pandas DataFrame.

The file contains two columns: **movie_id** and **plot**. The data is tab-separated, and the '\t' escape sequence is used as the separator.

In [3]:
df = pd.read_csv('../data/plot_summaries.tsv', sep='\t', names=['movie_id','plot'])

Review the first few rows of data to get an overview of how the data is structured.

In [4]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)

df.head(5)

movie_id  \
0  23890098   
1  31186339   
2  20663735   
3   2231378   
4    595909   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

To check the number of rows and columns, use the `shape` property.

In [5]:
df.shape

(42303, 2)

Now examine the metadata. The [dataset documentation](http://www.cs.cmu.edu/~ark/personas/data/README.txt) explains that the data contains nine fields. Load the data into a pandas DataFrame and specify the column names.

In [6]:
movie_meta_df = pd.read_csv('../data/movie.metadata.tsv', sep='\t', names=['movie_id','freebase_id','name','release_date','box_office_revenue','runtime','languages','countries','genres'])
movie_meta_df.head()

,movie_id,freebase_id,name,release_date,box_office_revenue,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science Fiction"", ""/m/03npn"": ""Horror"", ""/m/03k9fj"": ""Adventure"", ""/m/0fdjb"": ""Supernatural"", ""/m/02kdv5l"": ""Action"", ""/m/09zvmj"": ""Space western""}"
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey Mystery,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0"": ""Drama"", ""/m/0hj3n01"": ""Crime Drama""}"
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""Drama""}"
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic thriller"", ""/m/09blyk"": ""Psychological thriller""}"
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


Set the index to **movie_id**, which will make it easier to merge this dataset with the plot.

In [7]:
movie_meta_df.set_index('movie_id', inplace=True)

Because you only need the movie name and something to link this metadata to the plot (**movie_id**), drop the remaining columns.

In [8]:
movie_meta_df=movie_meta_df.drop(['freebase_id','release_date','box_office_revenue','runtime','languages','countries','genres'], axis=1)
movie_meta_df.head()

,name
movie_id,
975900,Ghosts of Mars
3196793,Getting Away with Murder: The JonBenét Ramsey Mystery
28463795,Brun bitter
9363483,White Of The Eye
261236,A Woman in Flames


## 3. Extracting topics
([Go to top](#Challenge-Lab-6.3:-Implementing-Topic-Extraction))

You must now decide if you are going to use Amazon Comprehend or the SageMaker NTM algorithm to extract your topics. Both will do a good job of giving you topics, but each has different data requirements.

Refer to the notebooks from labs 6.1 and 6.2 for any code snippets you might need for each solution. Experiment with the number of topics to see if you can get better results. 

Questions to address:

1. What data cleanup do you need to perform?

2. How many topics will give you the best results?

In [9]:
# Initialize a tokenizer and lemmatizer
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()

# Download stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = tokenizer.tokenize(text.lower())  # Tokenize and lowercase
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 2]  # Lemmatize
    return ' '.join(tokens)

# Apply preprocessing to the plot column
df['processed_plot'] = df['plot'].apply(preprocess_text)

# Preview the processed data
df[['movie_id', 'processed_plot']].head(5)

,movie_id,processed_plot
0,23890098,shlykov hard working taxi driver lyosha saxophonist develop bizarre love hate relationship despite prejudice realize different
1,31186339,nation panem consists wealthy capitol twelve poorer district punishment past rebellion district must provide boy girl age selected lottery annual hunger game tribute must fight death arena sole survivor rewarded fame wealth first reaping year old primrose everdeen chosen district older sister katniss volunteer take place peeta mellark baker son gave katniss bread starving district tribute katniss peeta taken capitol accompanied frequently drunk mentor past victor haymitch abernathy warns career tribute train intensively special academy almost always win interview caesar flickerman peeta unexpectedly reveals love katniss outraged believing ploy gain audience support sponsor may provide game gift food medicine tool however discovers peeta meant said televised game begin half tribute killed first minute katniss barely survives ignoring haymitch advice run away melee tempting supply weapon strewn front structure called cornucopia peeta form uneasy alliance four career later find katniss corner tree rue hiding nearby tree draw attention poisonous tracker jacker nest hanging branch katniss drop sleeping besieger scatter except glimmer killed insect hallucinating due tracker jacker venom katniss warned run away peeta rue care katniss couple day recovers meanwhile alliance gathered supply pile katniss rue draw destroys stockpile setting mine planted around furious cato kill boy assigned guard katniss run scene hears rue calling name find rue trapped release marvel tribute district throw spear katniss dodge spear causing stab rue stomach instead katniss shoot dead arrow comfort dying rue song afterward gather arranges flower around rue body televised spark riot rue district president snow summons seneca crane gamemaker express displeasure way game turning since katniss peeta presented public star crossed lover haymitch able convince crane make rule change avoid inciting riot announced tribute district win pair upon hearing katniss search peeta find infected sword wound leg portrays deeply love gain sponsor gift soup announcer proclaims feast thing survivor need provided peeta begs risk getting medicine katniss promise fall asleep head feast clove ambush pin clove gloat thresh district tribute kill clove overhearing tormenting katniss killing rue spare katniss time rue medicine work keeping peeta mobile foxface girl district dy eating nightlock berry stole peeta neither knew highly poisonous crane change time day arena late night unleashes pack hound like creature speed thing kill thresh force katniss peeta flee roof cornucopia encounter cato battle katniss wound cato arrow peeta hurl creature katniss shoot cato spare prolonged death peeta katniss apparently victorious rule change allowing two winner suddenly revoked peeta tell katniss shoot instead give half nightlock however commit suicide hastily proclaimed victor 74th hunger game haymitch warns katniss made powerful enemy display defiance peeta return district crane locked room bowl nightlock berry president snow considers situation
2,20663735,poovalli induchoodan sentenced six year prison life murdering classmate induchoodan son justice maranchery karunakara menon framed case manapally madhavan nambiar crony dysp sankaranarayanan take revenge idealist judge menon earlier given jail sentence manapally corruption case induchoodan achieved top rank indian civil service loses post manapally sudheeran saikumar enters list civil service trainee learn flashback ramakrishnan son moopil nair actually killed classmate six year pass manapally madhavan nambiar former state minister dead induchoodan rage gross injustice meted thus destroying promising life released prison induchoodan thwart manapally pavithran performing funeral ritual nambiar bharathapuzha many confrontation induchoodan manapally henchman follow induchooda

In [11]:
# Initialize S3 client
s3_client = boto3.client('s3')

# Define S3 bucket and file path
bucket = 'c125984a3128011l7679056t1w199902952632-labbucket-owxxnbun3jgz'
file_path = 'path/to/processed_plot_data.csv'

# Save the processed data to a local file
df[['processed_plot']].to_csv('processed_plot_data.csv', index=False)

# Upload the file to S3
s3_client.upload_file('processed_plot_data.csv', bucket, file_path)

In [13]:
comprehend_client = boto3.client('comprehend')

response = comprehend_client.start_topics_detection_job(
    JobName=f"top-job-{uuid.uuid1()}",
    InputDataConfig={
        'S3Uri': f's3://{bucket}/{file_path}',
        'InputFormat': 'ONE_DOC_PER_LINE'
    },
    OutputDataConfig={
        'S3Uri': f's3://{bucket}/output/',
    },
    DataAccessRoleArn='arn:aws:iam::199902952632:role/service-role/c125984a3128011l7679056t1w-ComprehendDataAccessRole-S5V4dviXZ8eE',
    NumberOfTopics=10  # Specify the number of topics
)

job_id = response['JobId']
print(f'Topic modeling job started: {job_id}')

Topic modeling job started: ccffbc1c4c551f56105e8d75e34f74f3


In [14]:
# Monitor the job status
job_status = comprehend_client.describe_topics_detection_job(JobId=job_id)['TopicsDetectionJobProperties']['JobStatus']
print(f'Job Status: {job_status}')

Job Status: IN_PROGRESS


In [15]:
import time

def check_job_status(job_id):
    while True:
        response = comprehend_client.describe_topics_detection_job(
            JobId=job_id
        )
        status = response['TopicsDetectionJobProperties']['JobStatus']
        print(f"Job Status: {status}")
        if status == 'COMPLETED':
            print("Job completed successfully!")
            break
        elif status == 'FAILED':
            raise Exception("The job failed.")
        time.sleep(30)  # Check every 30 seconds

# Start job and check status
check_job_status(job_id)

Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_P

In [16]:
import boto3

s3_client = boto3.client('s3')

def download_output_file(bucket, topics_comprehend_key, file_name):
    # Download the tar.gz file from the specified S3 bucket and prefix
    output_file = f'/tmp/{file_name}'
    s3_client.download_file(bucket, f'{topics_comprehend_key}', output_file)
    return output_file

In [17]:
import tarfile
import os

def extract_tar_gz(file_path, extract_to='/tmp'):
    with tarfile.open(file_path, "r:gz") as tar:
        tar.extractall(path=extract_to)
        print(f"Extracted files to {extract_to}")
        # List the files extracted
        extracted_files = os.listdir(extract_to)
        print(f"Extracted files: {extracted_files}")

        # Print out all files, including those in subdirectories
        for root, dirs, files in os.walk(extract_to):
            print(f"Directory: {root}")
            for file in files:
                print(f"File: {file}")

        return extracted_files

In [18]:
import shutil

# Function to search for and move CSV files
def move_csv_files(extract_to='/tmp', notebook_dir='../en_us'):
    found_files = []
    
    # Walk through all extracted directories and files
    for root, dirs, files in os.walk(extract_to):
        for file in files:
            if file in ['topic-terms.csv', 'doc-topics.csv']:
                full_file_path = os.path.join(root, file)
                found_files.append(full_file_path)
                # Move the file to the notebook directory
                shutil.copy(full_file_path, os.path.join(notebook_dir, file))
                print(f"Moved {file} to {notebook_dir}")

    if not found_files:
        print("No CSV files found.")

In [19]:
import csv

def print_topics(file_path):
    with open(file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)  # Skip header row
        for row in csv_reader:
            topic_index = row[0]    # Topic number
            term = row[1]           # Term associated with the topic
            score = row[2]          # Relevance score of the term
            print(f'Topic {topic_index}: {term} (Score: {score})')

In [20]:
response = comprehend_client.describe_topics_detection_job(
            JobId=job_id
        )
topic_comprehend_output_file = response['TopicsDetectionJobProperties']['OutputDataConfig']['S3Uri']
print(f'output filename: {topic_comprehend_output_file}')

topics_comprehend_bucket, topics_comprehend_key = topic_comprehend_output_file.replace("s3://", "").split("/", 1)
print(topics_comprehend_key)

output filename: s3://c125984a3128011l7679056t1w199902952632-labbucket-owxxnbun3jgz/output/199902952632-TOPICS-ccffbc1c4c551f56105e8d75e34f74f3/output/output.tar.gz
output/199902952632-TOPICS-ccffbc1c4c551f56105e8d75e34f74f3/output/output.tar.gz


In [21]:
# Define file name
file_name = 'output.tar.gz'

# Download tar.gz file
tar_gz_file = download_output_file(bucket, topics_comprehend_key, file_name)

# Extract the tar.gz file
extracted_files = extract_tar_gz(tar_gz_file)

# Move the CSV files to the notebook location
move_csv_files()

# Assuming topic-terms.csv is one of the extracted files, print topics
if 'topic-terms.csv' in extracted_files:
    print_topics('/tmp/topic-terms.csv')
else:
    print("topic-terms.csv not found!")

Extracted files to /tmp
Extracted files: ['.font-unix', '.X11-unix', '.ICE-unix', '.Test-unix', '.XIM-unix', 'output.tar.gz', 'doc-topics.csv', 'topic-terms.csv', 'systemd-private-bd1d0c9b686c4491a99dda395b4645f5-chronyd.service-UXJKz4', 'hsperfdata_role-agent', '.java_pid4139', 'jetty-localhost-9081-role-proxy-agent_war-_-any-10912996224794097439']
Directory: /tmp
File: output.tar.gz
File: doc-topics.csv
File: topic-terms.csv
File: .java_pid4139
Directory: /tmp/.font-unix
Directory: /tmp/.X11-unix
Directory: /tmp/.ICE-unix
Directory: /tmp/.Test-unix
Directory: /tmp/.XIM-unix
Directory: /tmp/hsperfdata_role-agent
File: 4139
Moved doc-topics.csv to ../en_us
Moved topic-terms.csv to ../en_us
Topic 000: money (Score: 0.0070251743)
Topic 000: gang (Score: 0.0046430626)
Topic 000: work (Score: 0.0056518908)
Topic 000: steal (Score: 0.0035103702)
Topic 000: make (Score: 0.0059012785)
Topic 000: police (Score: 0.004918466)
Topic 000: bank (Score: 0.0027779688)
Topic 000: job (Score: 0.0032056

# Congratulations!

You have completed this lab, and you can now end the lab by following the lab guide instructions.

*©2023 Amazon Web Services, Inc. or its affiliates. All rights reserved. This work may not be reproduced or redistributed, in whole or in part, without prior written permission from Amazon Web Services, Inc. Commercial copying, lending, or selling is prohibited. All trademarks are the property of their owners.*
